The easiest way to run an LLM without a GPU is using Ollama

In [52]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11435/v1/',
    api_key='ollama',
)

In [53]:
import json
from tqdm.auto import tqdm 
from elasticsearch import Elasticsearch

In [3]:
with open('../documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [32]:
es_client = Elasticsearch('http://localhost:9200')

In [33]:
es_client.info()

ObjectApiResponse({'name': '9623a47f4115', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'vpwpfoVGTl6VCsAAXv54Qw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [34]:
index_name = "course-questions"

In [35]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [36]:
query = 'I just disovered the course. Can I still join it?'

In [37]:
index_name = "course-questions"

In [45]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], #question is 3 times more important that text or question
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    print(response)
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [46]:
elastic_search(query)

{'took': 7, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 884, 'relation': 'eq'}, 'max_score': 79.96579, 'hits': [{'_index': 'course-questions', '_id': 'nXfBmJABHCNPSXoP6Fvu', '_score': 79.96579, '_ignored': ['text.keyword'], '_source': {'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': '3nfBmJABHCNPSXoP0lmD', '_score': 72.849266, '_source': {'text': "Yes, even if you don't register, you're still 

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Welcome to the course! Go to the course page (http

In [54]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.

        QUESTION: {question}

        CONTEXT: 
        {context}
        """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
    model='phi3',
    messages=[{"role":"user", "content":prompt}])
    return response.choices[0].message.content

In [55]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [56]:
rag(query)


{'took': 5, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 884, 'relation': 'eq'}, 'max_score': 79.96579, 'hits': [{'_index': 'course-questions', '_id': 'nXfBmJABHCNPSXoP6Fvu', '_score': 79.96579, '_ignored': ['text.keyword'], '_source': {'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': '3nfBmJABHCNPSXoP0lmD', '_score': 72.849266, '_source': {'text': "Yes, even if you don't register, you're still 

' Yes, even though I discovered/joined the course after it has already started, as a new student joining at the end of November who completes 2 out of my required projects and peer-review activities by their deadline, I would still be eligible for a certificate. Here are some steps to get me acclimated with accessing the resources:\n1) Go on this link http://mlzoomcamp.com/ which leads directly to our course syllabus page along with other important information about ML ZoomCamp (founded in 2021).\n2) Click into the "syllabus" section where you\'d find links like #homework, office hours for previous years and playlists related to {course year}. It would be best if I start watching these videos.\n3) Additionally, there is also a link that navigates directly to our course syllabus: http://mlzoomcamp.com/#syllabus which you can guide me on as an alternative navigation method into the resources for this ML ZoomCamp course.'